## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,lebu,CL,-37.6167,-73.6500,48.16,91,100,8.77,overcast clouds
1,1,ulaanbaatar,MN,47.9077,106.8832,50.13,71,0,0.00,clear sky
2,2,carnarvon,AU,-24.8667,113.6333,64.35,70,16,14.90,few clouds
3,3,tamasane,BW,-22.4184,27.4013,50.07,61,0,4.25,clear sky
4,4,abu kamal,SY,34.4506,40.9171,79.25,38,10,15.21,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 73
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,abu kamal,SY,34.4506,40.9171,79.25,38,10,15.21,clear sky
9,9,roebourne,AU,-20.7833,117.1333,73.58,50,45,10.49,scattered clouds
12,12,vanimo,PG,-2.6741,141.3028,81.12,82,28,6.76,scattered clouds
14,14,kavaratti,IN,10.5669,72.6420,83.01,73,100,18.57,light rain
16,16,port alfred,ZA,-33.5906,26.8910,77.18,28,100,8.01,overcast clouds
18,18,marawi,PH,7.9986,124.2928,75.54,80,87,0.87,overcast clouds
25,25,karratha,AU,-20.7377,116.8463,73.71,57,43,9.86,scattered clouds
29,29,avarua,CK,-21.2078,-159.7750,78.85,69,75,13.80,broken clouds
30,30,tautira,PF,-17.7333,-149.1500,78.13,68,46,19.04,scattered clouds
35,35,cabedelo,BR,-6.9811,-34.8339,75.87,81,8,8.84,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                252
City                   252
Country                252
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,abu kamal,SY,79.25,clear sky,34.4506,40.9171,
9,roebourne,AU,73.58,scattered clouds,-20.7833,117.1333,
12,vanimo,PG,81.12,scattered clouds,-2.6741,141.3028,
14,kavaratti,IN,83.01,light rain,10.5669,72.6420,
16,port alfred,ZA,77.18,overcast clouds,-33.5906,26.8910,
18,marawi,PH,75.54,overcast clouds,7.9986,124.2928,
25,karratha,AU,73.71,scattered clouds,-20.7377,116.8463,
29,avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
30,tautira,PF,78.13,scattered clouds,-17.7333,-149.1500,
35,cabedelo,BR,75.87,clear sky,-6.9811,-34.8339,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

print("Data Retrieval Complete      ")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Data Retrieval Complete      


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(20)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,abu kamal,SY,79.25,clear sky,34.4506,40.9171,البوكمال
1,9,roebourne,AU,73.58,scattered clouds,-20.7833,117.1333,Latitude20 Roebourne Village
2,12,vanimo,PG,81.12,scattered clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
3,14,kavaratti,IN,83.01,light rain,10.5669,72.6420,Hotels in Lakshadweep Islands
4,16,port alfred,ZA,77.18,overcast clouds,-33.5906,26.8910,The Halyards Hotel
5,18,marawi,PH,75.54,overcast clouds,7.9986,124.2928,Derogongan Residence
6,25,karratha,AU,73.71,scattered clouds,-20.7377,116.8463,ibis Styles Karratha
7,29,avarua,CK,78.85,broken clouds,-21.2078,-159.7750,Paradise Inn
8,30,tautira,PF,78.13,scattered clouds,-17.7333,-149.1500,Pension Oaoa
9,35,cabedelo,BR,75.87,clear sky,-6.9811,-34.8339,Pousada Camboinha


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))